# 本周学习总结

## 学习周期
**周次**：第5周  
**学习内容**：Introduction to Economic Modeling and Data Science
 -    Data Science Tools

## Intermediate Plotting

### **高级绘图功能**
- **Twin axis**：同一图表显示不同尺度的数据
- **Annotations**：添加文本和箭头标注
- **Subplots**：复杂布局

```python
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 创建数据
x = np.linspace(0, 2*np.pi, 100)
y1 = np.sin(x)
y2 = np.cos(x) * 100  # 不同尺度

# Twin axis（双Y轴）
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(x, y1, 'b-', label='sin(x)')
ax2.plot(x, y2, 'r-', label='100*cos(x)')

ax1.set_xlabel('x')
ax1.set_ylabel('sin(x)', color='b')
ax2.set_ylabel('100*cos(x)', color='r')
plt.show()

# Annotations（标注）
fig, ax = plt.subplots()
ax.plot(x, y1)
ax.annotate('Maximum', xy=(np.pi/2, 1), xytext=(np.pi/2, 0.5),
            arrowprops=dict(facecolor='black', shrink=0.05))
plt.show()

# Complex subplots（复杂子图布局）
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes[0, 0].plot(x, y1)
axes[0, 1].plot(x, y2)
axes[1, 0].scatter(x[::5], y1[::5])
axes[1, 1].hist(y1, bins=20)
plt.tight_layout()
plt.show()
```

### **样式定制**
- **Linestyles**：线型（`'-'`, `'--'`, `':'`）
- **Markers**：标记（`'o'`, `'^'`, `'s'`）
- **Colors**：颜色（`'b'`, `'g'`, `'#FF5733'`）

```python
# 自定义样式
fig, ax = plt.subplots()
ax.plot(x, y1, linestyle='--', marker='o', color='#1f77b4',
        markersize=5, linewidth=2, label='sin(x)')
ax.legend()
plt.show()
```

### **动画**
- **FuncAnimation**：创建动态图表

```python
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
line, = ax.plot([], [], 'r-')

def init():
    line.set_data([], [])
    return line,

def animate(i):
    x = np.linspace(0, 2*np.pi, 100)
    y = np.sin(x + i/10)
    line.set_data(x, y)
    return line,

ani = FuncAnimation(fig, animate, init_func=init, frames=100,
                    interval=50, blit=True)
plt.show()
```

## Maps

### **GeoPandas基础**
- **GeoDataFrame**：包含几何列（**geometry**）
-  **`.plot()`**  ：绘制地图

```python
import geopandas as gpd
import pandas as pd

# 读取shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# 简单地图
world.plot(figsize=(10, 6))
plt.show()

# 按人口着色
world.plot(column='gdp_md_est', cmap='OrRd', legend=True,
           figsize=(10, 6))
plt.title('GDP by Country')
plt.show()

# 添加边界
world.boundary.plot(figsize=(10, 6))
plt.show()
```


### **空间连接（Spatial Join）**
-  **`.sjoin()`**  ：基于空间关系的合并

```python
# 创建点数据
cities = pd.DataFrame({
    'name': ['New York', 'Paris', 'Tokyo'],
    'longitude': [-74.0060, 2.3522, 139.6917],
    'latitude': [40.7128, 48.8566, 35.6762]
})
cities_gdf = gpd.GeoDataFrame(
    cities, geometry=gpd.points_from_xy(cities.longitude, cities.latitude))

# 空间连接（点落在哪个国家）
cities_with_country = gpd.sjoin(
    cities_gdf, world[['name', 'geometry']], how='left')
print(cities_with_country)
```

### **交互式地图（Folium）**
- **folium.Map()**：创建Leaflet地图
- **folium.Marker()**：添加标记
- **folium.Choropleth()**：添加Choropleth图层

```python
import folium

# 创建基础地图
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# 添加标记
folium.Marker(
    location=[40.7128, -74.0060],
    popup='New York City',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# 显示地图
m.save('map.html')
```


### **投影（Projections）**
-  **`.to_crs()`**  ：转换坐标参考系统

```python
# 查看当前CRS
print(world.crs)

# 转换为Web Mercator（用于在线地图）
world_web = world.to_crs('EPSG:3857')
print(world_web.crs)
```

## Visualization Rules

### **图表选择原则**
- **Line chart**：展示趋势
- **Scatter plot**：展示关系
- **Bar chart**：比较类别
- **Histogram**：分布展示
- **Box plot**：分布和异常值

```python
import seaborn as sns
import matplotlib.pyplot as plt

# 示例数据
tips = sns.load_dataset('tips')

# Rule 1: 选择合适图表
# 趋势 - line plot
tips.groupby('size')['tip'].mean().plot(kind='line')
plt.show()

# 关系 - scatter plot
sns.scatterplot(data=tips, x='total_bill', y='tip')
plt.show()

# 比较 - bar chart
sns.barplot(data=tips, x='day', y='total_bill')
plt.show()

# 分布 - histogram
sns.histplot(tips['total_bill'], bins=30, kde=True)
plt.show()

# 分布+异常值 - box plot
sns.boxplot(data=tips, x='day', y='total_bill')
plt.show()
```

### **视觉感知原则**
- **避免3D图表**：扭曲数据感知
- **合理使用颜色**：不超过6-8种颜色
- **注意色盲友好**：使用ColorBrewer调色板

```python
# 色盲友好的调色板
palette = sns.color_palette('colorblind')
sns.set_palette(palette)

# 避免3D（不好）
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')  # 扭曲比例

# 2D更好
fig, ax = plt.subplots()
ax.scatter(tips['total_bill'], tips['tip'])
plt.show()
```

### **标注和注释**
- **清晰的标题和标签**
- **图例位置优化**
- **重要数据点标注**

```python
fig, ax = plt.subplots()
ax.plot(tips['total_bill'], tips['tip'], 'o')

# 清晰标签
ax.set_xlabel('Total Bill ($)', fontsize=12)
ax.set_ylabel('Tip ($)', fontsize=12)
ax.set_title('Tip vs Total Bill', fontsize=14)

# 图例
ax.legend(['Observations'], loc='upper left')

plt.show()
```

### **图表垃圾（Chart Junk）**
- **减少非必要元素**：网格线、背景、边框
- **最大化信息墨水比**

```python
# 好例子：简洁
fig, ax = plt.subplots()
ax.plot(tips['total_bill'], tips['tip'], 'o', markersize=3)
ax.set_xlabel('Total Bill')
ax.set_ylabel('Tip')
ax.spines['top'].set_visible(False)  # 移除上边框
ax.spines['right'].set_visible(False)  # 移除右边框
plt.show()

# 坏例子：太多装饰
fig, ax = plt.subplots()
ax.plot(tips['total_bill'], tips['tip'], 'o')
ax.grid(True, linestyle='--', alpha=0.7)
ax.set_xlabel('Total Bill')
ax.set_ylabel('Tip')
# 额外的装饰元素...
plt.show()
```

### **比例和尺度**
- **Y轴应从0开始**（除非有明确理由）
- **避免过度放大差异**

```python
# 好：Y轴从0开始
fig, ax = plt.subplots()
ax.bar(tips['day'], tips['total_bill'].groupby(tips['day']).mean())
ax.set_ylim(0, 30)  # 明确从0开始
plt.show()

# 坏：Y轴不从0开始（放大差异）
fig, ax = plt.subplots()
ax.bar(tips['day'], tips['total_bill'].groupby(tips['day']).mean())
ax.set_ylim(15, 25)  # 不从0开始，误导
plt.show()
```


## Regression


### **Statsmodels回归**
- **OLS**：`sm.OLS()`
- **结果摘要**：`summary()`提供统计检验

```python
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

# 加载数据
url = "https://datascience.quantecon.org/assets/data/employment.csv"
emp = pd.read_csv(url)

# 方法1: 公式接口
model = smf.ols('earnings ~ education + age + 1', data=emp)
results = model.fit()
print(results.summary())

# 方法2: 数组接口
y = emp['earnings']
X = emp[['education', 'age']]
X = sm.add_constant(X)  # 添加常数项

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())
```

### **回归诊断**
- **残差分析**：检查模型假设
- **异方差性**：使用**robust standard errors**

```python
# 残差图
fig, ax = plt.subplots()
ax.scatter(results.fittedvalues, results.resid)
ax.axhline(0, color='red', linestyle='--')
ax.set_xlabel('Fitted Values')
ax.set_ylabel('Residuals')
ax.set_title('Residual Plot')
plt.show()

# Robust standard errors
results_robust = model.fit(cov_type='HC3')
print(results_robust.summary())
```

### **Scikit-learn回归**
- **LinearRegression**：预测导向
- **Ridge/Lasso**：正则化

```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

# 准备数据
X = emp[['education', 'age']]
y = emp['earnings']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 训练模型
lr = LinearRegression()
lr.fit(X_train, y_train)

# 预测和评估
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse:.2f}")
```

### **多项式回归**
- **PolynomialFeatures**：创建多项式特征

```python
from sklearn.preprocessing import PolynomialFeatures

# 创建二次项
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X[['age']])

# 拟合多项式回归
poly_reg = LinearRegression()
poly_reg.fit(X_poly, y)

# 可视化
age_grid = np.linspace(X['age'].min(), X['age'].max(), 100)
age_poly_grid = poly.transform(age_grid.reshape(-1, 1))
pred_grid = poly_reg.predict(age_poly_grid)

fig, ax = plt.subplots()
ax.scatter(X['age'], y, alpha=0.5, s=10)
ax.plot(age_grid, pred_grid, color='red')
ax.set_xlabel('Age')
ax.set_ylabel('Earnings')
plt.show()
```

## Classification

### **逻辑回归（Logistic Regression）**
- **LogisticRegression**：二分类和多分类
- **概率输出**：`predict_proba()`

```python
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# 加载数据
url = "https://datascience.quantecon.org/assets/data/employment.csv"
emp = pd.read_csv(url)

# 创建二元目标变量（高/低收入）
emp['high_earner'] = (emp['earnings'] > emp['earnings'].median()).astype(int)

# 准备特征
X = emp[['education', 'age', 'edu_expense_ratio']]
y = emp['high_earner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 训练逻辑回归
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# 预测和评估
y_pred = log_reg.predict(X_test)
y_pred_proba = log_reg.predict_proba(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(classification_report(y_test, y_pred))
```


### **决策树（Decision Trees）**
- **DecisionTreeClassifier**：非线性分类
- **可视化树结构**

```python
from sklearn.tree import DecisionTreeClassifier, plot_tree

# 训练决策树
tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)

# 可视化
fig, ax = plt.subplots(figsize=(12, 8))
plot_tree(tree, feature_names=X.columns, class_names=['Low', 'High'],
          filled=True, ax=ax)
plt.show()

# 预测
y_pred_tree = tree.predict(X_test)
print(f"Tree Accuracy: {accuracy_score(y_test, y_pred_tree):.3f}")
```

### **模型评估**
- **交叉验证（Cross-validation）**
- **混淆矩阵（Confusion Matrix）**
- **ROC曲线和AUC**

```python
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc, confusion_matrix

# 交叉验证
cv_scores = cross_val_score(LogisticRegression(), X, y, cv=5)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV score: {cv_scores.mean():.3f}")

# ROC曲线
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_proba = log_reg.predict_proba(X_test)[:, 1]

fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

fig, ax = plt.subplots()
ax.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver Operating Characteristic')
ax.legend()
plt.show()

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)
print(cm)
```



### **特征重要性**
- **系数分析**（逻辑回归）
- **特征重要性**（决策树）

```python
# 逻辑回归系数
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'coefficient': log_reg.coef_[0]
})
print(feature_importance.sort_values('coefficient', ascending=False))

# 决策树特征重要性
tree_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': tree.feature_importances_
})
print(tree_importance.sort_values('importance', ascending=False))
```